# 01. Apache Iceberg 아키텍처 개요

이 노트북에서는 Apache Iceberg의 **3-Layer 아키텍처**를 실제 파일 구조를 통해 확인합니다.

## Iceberg 3-Layer 아키텍처

Apache Iceberg 테이블은 세 개의 계층으로 구성됩니다.

```
┌─────────────────────────────────┐
│         Catalog Layer           │
│  (테이블 이름 → 현재 metadata   │
│   파일 위치를 가리키는 포인터)    │
└──────────────┬──────────────────┘
               │
               ▼
┌─────────────────────────────────┐
│        Metadata Layer           │
│  metadata.json → manifest list  │
│  → manifest files               │
│  (스냅샷, 스키마, 파티션 정보,   │
│   파일 수준 통계 등)             │
└──────────────┬──────────────────┘
               │
               ▼
┌─────────────────────────────────┐
│          Data Layer             │
│  실제 데이터가 저장된            │
│  Parquet / ORC / Avro 파일들    │
└─────────────────────────────────┘
```

### 각 계층의 역할

| 계층 | 구성 요소 | 역할 |
|------|-----------|------|
| **Catalog Layer** | Hadoop FS / Hive Metastore / REST 등 | 테이블 이름을 현재 metadata.json 파일 경로로 매핑 |
| **Metadata Layer** | metadata.json, snap-*.avro, manifest-*.avro | 테이블 스냅샷, 스키마, 파티션 정보, 파일 수준 통계 관리 |
| **Data Layer** | .parquet 파일들 | 실제 레코드가 저장된 데이터 파일 |

## 환경 설정

In [8]:
import sys
sys.path.append('..')

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, count_files, total_size

In [9]:
spark = create_spark_session("ArchOverview")

Spark + Iceberg 세션 준비 완료 (warehouse: file:///home/jovyan/data/warehouse)


## 테이블 생성

`PARTITIONED BY (months(order_date))`를 사용하여 월 단위 파티션을 설정합니다.  
이것은 Iceberg의 **Hidden Partitioning** 기능으로, 사용자가 파티션 컬럼을 직접 관리할 필요가 없습니다.

In [10]:
spark.sql("CREATE DATABASE IF NOT EXISTS demo.lab")
spark.sql("DROP TABLE IF EXISTS demo.lab.arch_orders")

spark.sql("""
    CREATE TABLE demo.lab.arch_orders (
        order_id     BIGINT,
        customer_id  BIGINT,
        product_name STRING,
        order_date   DATE,
        amount       DOUBLE,
        status       STRING
    )
    USING iceberg
    PARTITIONED BY (months(order_date))
""")

print("테이블 생성 완료: demo.lab.arch_orders")

테이블 생성 완료: demo.lab.arch_orders


## 데이터 삽입

여러 번의 배치 적재를 시뮬레이션합니다.
**각 append마다 새로운 스냅샷, manifest list, manifest file**이 생성되므로 Metadata Layer가 어떻게 쌓이는지 관찰할 수 있습니다.

In [11]:
# 4번의 배치 적재 → 4개의 스냅샷, 각각 새로운 manifest file 생성
for i in range(4):
    orders = generate_orders(num_records=200, seed=i, id_offset=i*200+1)
    df = to_spark_df(spark, orders)
    df.writeTo("demo.lab.arch_orders").append()
    print(f"배치 {i+1}/4 완료 (200건 삽입)")

count = spark.sql("SELECT COUNT(*) AS cnt FROM demo.lab.arch_orders").collect()[0]["cnt"]
print(f"\n총 레코드: {count}건")

배치 1/4 완료 (200건 삽입)
배치 2/4 완료 (200건 삽입)
배치 3/4 완료 (200건 삽입)
배치 4/4 완료 (200건 삽입)

총 레코드: 800건


## Warehouse 디렉토리 구조 탐색

`show_tree`로 Iceberg 테이블이 파일 시스템에 어떤 구조로 저장되는지 확인합니다.

In [12]:
TABLE_PATH = "/home/jovyan/data/warehouse/lab/arch_orders"

show_tree(TABLE_PATH)

├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00000-11-5decdb65-aafa-48cf-b0b7-17fe7d84e297-00001.parquet  (2.9 KB)
│   │   ├── 00000-17-9ba26da7-af04-4847-aa9c-21c79fbb45f2-00001.parquet  (2.9 KB)
│   │   ├── 00000-23-3eeb50fc-6dcd-48b8-bbaf-fe8714f1ad17-00001.parquet  (2.9 KB)
│   │   └── 00000-5-bc643c3e-960f-4d5e-abf7-a34c45cbf6cc-00001.parquet  (2.9 KB)
│   ├── order_date_month=2024-02/
│   │   ├── 00000-11-5decdb65-aafa-48cf-b0b7-17fe7d84e297-00002.parquet  (2.9 KB)
│   │   ├── 00000-17-9ba26da7-af04-4847-aa9c-21c79fbb45f2-00002.parquet  (2.9 KB)
│   │   ├── 00000-23-3eeb50fc-6dcd-48b8-bbaf-fe8714f1ad17-00002.parquet  (2.9 KB)
│   │   └── 00000-5-bc643c3e-960f-4d5e-abf7-a34c45cbf6cc-00002.parquet  (2.9 KB)
│   └── order_date_month=2024-03/
│       ├── 00000-11-5decdb65-aafa-48cf-b0b7-17fe7d84e297-00003.parquet  (2.9 KB)
│       ├── 00000-17-9ba26da7-af04-4847-aa9c-21c79fbb45f2-00003.parquet  (2.8 KB)
│       ├── 00000-23-3eeb50fc-6dcd-48b8-bbaf-fe8714f1ad17-00003.parque

## 파일 통계

In [13]:
parquet_count = count_files(TABLE_PATH, ext=".parquet")
json_count = count_files(TABLE_PATH, ext=".json")
avro_count = count_files(TABLE_PATH, ext=".avro")
total = total_size(TABLE_PATH)

print(f"Parquet 파일 (Data Layer):    {parquet_count}개")
print(f"JSON 파일 (Metadata Layer):   {json_count}개")
print(f"Avro 파일 (Metadata Layer):   {avro_count}개")
print(f"전체 크기:                    {total / 1024:.1f} KB")

Parquet 파일 (Data Layer):    12개
JSON 파일 (Metadata Layer):   5개
Avro 파일 (Metadata Layer):   8개
전체 크기:                    98.2 KB


## 관찰 포인트

디렉토리 구조를 보면 크게 두 영역으로 나뉩니다:

### Data Layer (`data/` 디렉토리)
- `data/` 안에 파티션 디렉토리들이 존재 (예: `order_date_month=2024-01/`)
- 각 파티션 안에 `.parquet` 파일 = **실제 데이터**
- **4번의 append로 파티션당 최대 4개의 parquet 파일**이 생성됨

### Metadata Layer (`metadata/` 디렉토리)

| 파일 | 역할 |
|------|------|
| `v1~v5.metadata.json` | 테이블 생성(v1) + 4번 append(v2~v5). 가장 최신 버전이 현재 상태 |
| `snap-*.avro` (4개) | **Manifest List** — 각 스냅샷이 어떤 manifest file들을 참조하는지 |
| `*-m0.avro` (4개) | **Manifest File** — 각 append에서 추가된 parquet 파일 목록 + 통계 |

### 핵심: 계층 구조

```
metadata.json (현재 스냅샷 ID를 가리킴)
  └─▶ snap-*.avro (Manifest List)  ← 스냅샷 4는 manifest 4개를 참조
        ├─▶ manifest-1.avro  → [parquet-1a, parquet-1b, parquet-1c]  (배치 1)
        ├─▶ manifest-2.avro  → [parquet-2a, parquet-2b, parquet-2c]  (배치 2)
        ├─▶ manifest-3.avro  → [parquet-3a, parquet-3b, parquet-3c]  (배치 3)
        └─▶ manifest-4.avro  → [parquet-4a, parquet-4b, parquet-4c]  (배치 4)
```

**최신 스냅샷의 manifest list가 이전 배치의 manifest file까지 모두 포함**합니다.
이렇게 해야 "현재 테이블의 전체 데이터"를 알 수 있기 때문입니다.

### Catalog Layer
- Hadoop catalog의 경우, `version-hint.text` 파일이 현재 metadata.json의 버전 번호를 가리킵니다
- 이것이 Catalog Layer의 "포인터" 역할

---

**다음 노트북에서 이 파일들의 내부를 분석합니다.** Data Layer의 Parquet 파일 내부 구조와 Metadata Layer의 JSON/Avro 파일 내용을 직접 읽어보겠습니다.

In [14]:
spark.stop()
print("Spark 세션 종료")

Spark 세션 종료
